<a href="https://colab.research.google.com/github/RikiyaT/MusicRecSys/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spotipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 kB 8.5 MB/s eta 0:00:00


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

from skimage import io
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime


from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
spotify_data=pd.read_csv("SpotifyFeatures.csv")
spotify_data.head()

In [ ]:
spotify_features_df=spotify_data
genre_OHE=pd.get_dummies(spotify_features_df.genre)
key_OHE=pd.get_dummies(spotify_features_df.key)

In [ ]:
scaled_features=MinMaxScaler().fit_transform([
    spotify_features_df['acousticness'].values, 
    spotify_features_df['danceability'].values, 
    spotify_features_df['duration_ms'].values, 
    spotify_features_df['energy'].values, 
    spotify_features_df['instrumentalness'].values, 
    spotify_features_df['liveness'].values, 
    spotify_features_df['loudness'].values, 
    spotify_features_df['speechiness'].values, 
    spotify_features_df['tempo'].values, 
    spotify_features_df['valence'].values
])

In [ ]:
spotify_features_df[['acousticness','danceability','duration_ms','energy','instrumentalness','liveness','loudness','speechiness','tempo','valence']] = scaled_features.T

In [ ]:
spotify_features_df = spotify_features_df.drop('genre', axis = 1)
spotify_features_df = spotify_features_df.drop('artist_name', axis = 1)
spotify_features_df = spotify_features_df.drop('track_name', axis = 1)
spotify_features_df = spotify_features_df.drop('popularity',axis = 1)
spotify_features_df = spotify_features_df.drop('key', axis = 1)
spotify_features_df = spotify_features_df.drop('mode', axis = 1)
spotify_features_df = spotify_features_df.drop('time_signature', axis = 1)

In [ ]:
spotify_features_df = spotify_features_df.join(genre_OHE)
spotify_features_df = spotify_features_df.join(key_OHE)

In [ ]:
spotify_features_df.head()